# Import All Libaries

In [1]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
ensure_pyterrier_is_loaded()

import pyterrier as pt
from jnius import autoclass


# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


# Load the Dataset and the Index

In [2]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
dataset = 'ir-acl-anthology-20240504-training'
pt_dataset = pt.get_dataset(f'irds:ir-lab-sose-2024/{dataset}')
#bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

### Bo1 Query Expansion

In [3]:
bo1_expansion = bm25 >> pt.rewrite.Bo1QueryExpansion(index)
# build final pipeline for retrieval
bm25_bo1 = bo1_expansion >> bm25

### Evaluation

In [4]:
pt.Experiment(
    retr_systems=[bm25, bm25_bo1],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25', 'BM25_Bo1'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']
)

,name,recall_1000,ndcg_cut_5,ndcg_cut.10,recip_rank
0,BM25,0.825376,0.393650,0.374041,0.579877
1,BM25_Bo1,0.833643,0.381675,0.378220,0.571535
